# 卷积神经网络
上面我们介绍的线性回归和多层感知机等均为全连接网络，但是全连接网络的两个缺点却对图片一类的数据很不友好：
1. 参数量过大： 全连接层在处理输入（h1,w1,c1）输出（h2,w2,c2）时， 需要的参数量为h1\*w1\*c1\*h2\*w2\*c2, 对于200\*200\*3大小的图片，单层的参数量就达到了14400000000。
2. 没有办法识别空间局部特征，图片往往有空间相对信息，因此上下左右相邻的像素组合比展平的元素更有意义，同时缩放，平移，旋转不会改变这个区域的特征，但是全连接层则需要单独检测。

为了解决上述问题，同时基于生物感受野的思想，卷积神经网络横空出世。
在卷积神经网络中，影响输出元素 x 的前向计算的所有可能输入区域（可能大于输入的实际尺寸）叫做 x 的感受野（receptive field）

通过局部连接，权重共享，汇聚的结构，卷积神经网络解决了上述两个问题并不断发展，下面我们来讲述一下卷积神经网络的组成和技术细节。

## 卷积神经网络组成
卷积神经网络主要有卷积层，池化层，归一化层，全连接层组成。

### 卷积层
首先卷积层中最重要的就是卷积，但是实际上卷积层是做的互相关运算，而不是卷积运算。基于二者是旋转180度的关系，同时卷积层的参数是可学习的，因此我们可以直接使用互相关运算等效替代卷积运算并减少计算量。

卷积运算公式：
<div align=center>

![Untitled/Untitled.png](Pics/conv.png)

</div>
互相关运算公式：
<div align=center>

![Untitled/Untitled.png](Pics/corr.png)

</div>

在对图像做完卷积之后，我们还需要增加一个偏置b到最后的运算结果上，从而和之前的全连接层保持一致，记录与特征无关的平移，如下所示：

<div align=center>

![Untitled/Untitled.png](Pics/convmulc.png)

</div>

<div align=center>

![Untitled/Untitled.png](Pics/convbias.png)

</div>


卷积中的可选操作

在卷积层中除了进行卷积的操作，还可以进行一些其他的操作保证更好地利用原图/特征图上的信息。
* padding：为了保证原图/特征图的边缘也能多次参与和卷积核的运算并保证输入输出大小的不变性，人们采取将周边铺上0元素的方法，我们记单边增加了p列0元素。
* 步长：是指卷积核在滑动时的时间间隔。有时图片的尺寸较大，而我们为了减少计算量，并尽可能多覆盖相同多的特征，就会采用带步长的卷积，我们记步长为s
* 输入输出尺寸对应：设输入单边长度为lin，那么对应的输出单边长度为lout，而卷积核的单边长度为k
<div align=center>

![Untitled/Untitled.png](Pics/convsize.gif)

</div>

除此之外，还有空洞卷积，可变形卷积等其他卷积的变种可以用来进行卷积的操作。

### Pooling层
汇聚层的作用是进行特征选择，降低特征数量，从而减少参数数量，同时可以对一些小的局部形态改变保持不变性，并拥有更大的感受野。但是过大的汇聚层会造成信息的损失。

汇聚层的反向传播：和卷积层的反向传播不同，汇聚层的输入和输出不一致，因此，我们反向传播时需要考虑怎样将误差值从输出传播到输入。

如果下采样是最大汇聚，误差项 𝛿(𝑙+1,𝑝) 中每个值会直接传递到上一层对应区域中的最大值所对应的神经元，该区域中其他神经元的误差项都设为0．如果下采样是平均汇聚，误差项𝛿(𝑙+1,𝑝) 中每个值会被平均分配到上一层对应区域中的所有神经元上
<div align=center>

![Untitled/Untitled.png](Pics/poolgrad.png)

</div>

### 归一化层
通常，为了统一输入数据的量纲，我们会将输入数据进行归一化，从而可以避免量纲这种无关条件对神经网络学习的影响。
但是光对输入进行归一化是不够的，在神经网络学习过程中，内部每一层输出也会不断发生变化，如果限制变化范围，就会使得后层神经层需要不断适应浅层神经层的输出，从而带来训练不稳定和训练速度慢的问题。为了解决这个问题，我们在神经网络的输出之后引入各种类型的归一化操作。

**归一化层的作用就是通过规范每一层神经网络输出的分布，限制前层输出分布变化带来的后层输入的不稳定，从而引起的链式反应。**

#### 批归一化层
我们通常认为卷积层输出某一个通道代表了一类特征，因此批归一化通过将一个通道的所有特征图，使用样本统计量进行归一化并再一次进行仿射变换，获得变换后输出。也就是说，对于（N, C, H, W）的输入，其均值和方差的形状均为（1, C, 1, 1）。批量归一化主要有以下优势：

    1. 在使用sigmoid/tanh激活函数时，限制神经元的输出范围，使得输出落在激活函数梯度较大的区域，避免梯度消失的情况，同样可以减少梯度爆炸的情况
    2. 在深层网络中限制浅层神经网络输出的变化范围，避免深层神经元对变化大的浅层神经元输出的不断适应导致的训练不稳定问题
    4. 起到正则化的效果，因为限制输出的范围，从而将输出空间大大减小，


其推导公式如下：

#### 1.对全连接层做批量归一化
位置：全连接层中的仿射变换和激活函数之间。
**全连接：**
$$
\boldsymbol{x} = \boldsymbol{W\boldsymbol{u} + \boldsymbol{b}} \\
 output =\phi(\boldsymbol{x})
 $$


**批量归一化：**
$$
output=\phi(\text{BN}(\boldsymbol{x}))$$


$$
\boldsymbol{y}^{(i)} = \text{BN}(\boldsymbol{x}^{(i)})
$$


$$
\boldsymbol{\mu}_\mathcal{B} \leftarrow \frac{1}{m}\sum_{i = 1}^{m} \boldsymbol{x}^{(i)},
$$
$$
\boldsymbol{\sigma}_\mathcal{B}^2 \leftarrow \frac{1}{m} \sum_{i=1}^{m}(\boldsymbol{x}^{(i)} - \boldsymbol{\mu}_\mathcal{B})^2,
$$


$$
\hat{\boldsymbol{x}}^{(i)} \leftarrow \frac{\boldsymbol{x}^{(i)} - \boldsymbol{\mu}_\mathcal{B}}{\sqrt{\boldsymbol{\sigma}_\mathcal{B}^2 + \epsilon}},
$$

这⾥ϵ > 0是个很小的常数，保证分母大于0


$$
{\boldsymbol{y}}^{(i)} \leftarrow \boldsymbol{\gamma} \odot
\hat{\boldsymbol{x}}^{(i)} + \boldsymbol{\beta}.
$$


引入可学习参数：拉伸参数γ和偏移参数β。若$\boldsymbol{\gamma} = \sqrt{\boldsymbol{\sigma}_\mathcal{B}^2 + \epsilon}$和$\boldsymbol{\beta} = \boldsymbol{\mu}_\mathcal{B}$，批量归一化无效。

### 2.对卷积层做批量归⼀化
位置：卷积计算之后、应⽤激活函数之前。
如果卷积计算输出多个通道，我们需要对这些通道的输出分别做批量归一化，且每个通道都拥有独立的拉伸和偏移参数。
计算：对单通道，batchsize=m,对该通道中m×w×h个元素同时做批量归一化,使用相同的均值和方差。

### 3.预测时的批量归⼀化
训练：以batch为单位,对每个batch计算均值和方差。
预测：用移动平均估算整个训练数据集的样本均值和方差作为测试时的均值和方差。

### 从零实现

In [ ]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F
import torchvision
import sys
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batch_norm(is_training, X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 判断当前模式是训练模式还是预测模式
    if not is_training:
        # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 使用全连接层的情况，计算特征维上的均值和方差
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。这里我们需要保持
            # X的形状以便后面可以做广播运算
            mean = X.mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
            var = ((X - mean) ** 2).mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
        # 训练模式下用当前的均值和方差做标准化
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # 更新移动平均的均值和方差
        moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
        moving_var = momentum * moving_var + (1.0 - momentum) * var
    Y = gamma * X_hat + beta  # 拉伸和偏移
    return Y, moving_mean, moving_var

In [ ]:
class BatchNorm(nn.Module):
    def __init__(self, num_features, num_dims):
        super(BatchNorm, self).__init__()
        if num_dims == 2:
            shape = (1, num_features) #全连接层输出神经元
        else:
            shape = (1, num_features, 1, 1)  #通道数
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成0和1
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 不参与求梯度和迭代的变量，全在内存上初始化成0
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.zeros(shape)

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # 保存更新过的moving_mean和moving_var, Module实例的traning属性默认为true, 调用.eval()后设成false
        Y, self.moving_mean, self.moving_var = batch_norm(self.training,
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9)
        return Y

#### 在Lenet中的应用

In [ ]:
net = nn.Sequential(
            nn.Conv2d(1, 6, 5), # in_channels, out_channels, kernel_size
            BatchNorm(6, num_dims=4),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2), # kernel_size, stride
            nn.Conv2d(6, 16, 5),
            BatchNorm(16, num_dims=4),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2),
            d2l.FlattenLayer(),
            nn.Linear(16*4*4, 120),
            BatchNorm(120, num_dims=2),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            BatchNorm(84, num_dims=2),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )
print(net)

#### 简洁实现

In [ ]:
net = nn.Sequential(
            nn.Conv2d(1, 6, 5), # in_channels, out_channels, kernel_size
            nn.BatchNorm2d(6),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2), # kernel_size, stride
            nn.Conv2d(6, 16, 5),
            nn.BatchNorm2d(16),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2),
            d2l.FlattenLayer(),
            nn.Linear(16*4*4, 120),
            nn.BatchNorm1d(120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.BatchNorm1d(84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )

optimizer = torch.optim.Adam(net.parameters(), lr=lr)
train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

### 卷积神经网络的整体结构
通过组合卷积层，汇聚层，全连接层，卷积层用来识别图像里的空间模式，之后池化层用来降低卷积层对位置的敏感性,最后全连接层对物体进行分类。我们就可以获得以下一般的卷积神经网络结构：
<div align=center>

![Untitled/Untitled.png](Pics/convstruct.png)

</div>

不过网络结构并不是一成不变的，而是根据需要不断去修改其中卷积的操作，层级的安排等等。我们现在已经越来越少见到卷积神经网络中的全连接层了，汇聚层的比例也在不断减少。

## 经典的卷积神经网络
通过学习一些经典的神经网络，我们可以学习到其中设计网络的思想

### Lenet

Lenet的网络结构也遵循了我们之前提到的（卷积+激活+池化）*N+全连接的公式。
* 卷积层块由两个这样的基本单位重复堆叠构成。在卷积层块中，每个卷积层都使用 5×5 的窗口，并在输出上使用sigmoid激活函数。第一个卷积层输出通道数为6，第二个卷积层输出通道数则增加到16。
* 全连接层块含3个全连接层。它们的输出个数分别是120、84和10，其中10为输出的类别个数。
<div align=center>

![Untitled/Untitled.png](Pics/lenet.png)

</div>

### Alexnet
相较于Lenet，Alexnet是一个更加现代的卷积神经网络，其使用更深的网络来提升网络的参数/表征空间，并用一些技巧来控制模型的复杂度。主要有以下特征：
1. 8层变换，其中有5层卷积和2层全连接隐藏层，以及1个全连接输出层。
2. 将sigmoid激活函数改成了更加简单的ReLU激活函数。
3. 用Dropout来控制全连接层的模型复杂度。
4. 引入数据增强，如翻转、裁剪和颜色变化，从而进一步扩大数据集来缓解过拟合。

<div align=center>

![Image Name](Pics/ALEX.png)

</div>

除此之外，Alexnet还使用全卷积的方式来减少显存的占用，以往，我们每生成一张输出特征图就需要使用和输入相同channels的卷积核进行卷积，在群卷积中，每次只需要使用输入channels/K的卷积核进行运算，并把最后结果concate起来，就能获得和原来相同大小的输出。
群卷积有助于减小参数的数量，但是其限制了每个卷积核对于通道的访问，从而限制了特征的组合。下图阐述了群卷积的运行：
<div align=center>

![gconv.png](Pics/gconv.png)
</div>

In [ ]:
import time
import torch
from torch import nn, optim
import torchvision
import numpy as np
import sys
import os
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 96, 11, 4), # in_channels, out_channels, kernel_size, stride, padding
            nn.ReLU(),
            nn.MaxPool2d(3, 2), # kernel_size, stride
            # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            # 连续3个卷积层，且使用更小的卷积窗口。除了最后的卷积层外，进一步增大了输出通道数。
            # 前两个卷积层后不使用池化层来减小输入的高和宽
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )
         # 这里全连接层的输出个数比LeNet中的大数倍。使用丢弃层来缓解过拟合
        self.fc = nn.Sequential(
            nn.Linear(256*5*5, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            #由于使用CPU镜像，精简网络，若为GPU镜像可添加该层
            #nn.Linear(4096, 4096),
            #nn.ReLU(),
            #nn.Dropout(0.5),

            # 输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
            nn.Linear(4096, 10),
        )

    def forward(self, img):

        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return output

### VGG
使用多个重复的模块进行叠加的网络结构。
* Block:数个相同的填充为1、窗口形状为 3×3 的卷积层,接上一个步幅为2、窗口形状为 2×2 的最大池化层。
* 卷积层保持输入的高和宽不变，而池化层则对其减半。
<div align=center>

![Image Name](Pics/VGG.png)

</div>

In [ ]:
## VGG11 的简单实现

def vgg_block(num_convs, in_channels, out_channels): #卷积层个数，输入通道数，输出通道数
    blk = []
    for i in range(num_convs):
        if i == 0:
            blk.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        else:
            blk.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1))
        blk.append(nn.ReLU())
    blk.append(nn.MaxPool2d(kernel_size=2, stride=2)) # 这里会使宽高减半
    return nn.Sequential(*blk)

conv_arch = ((1, 1, 64), (1, 64, 128), (2, 128, 256), (2, 256, 512), (2, 512, 512))
# 经过5个vgg_block, 宽高会减半5次, 变成 224/32 = 7
fc_features = 512 * 7 * 7 # c * w * h
fc_hidden_units = 4096 # 任意

def vgg(conv_arch, fc_features, fc_hidden_units=4096):
    net = nn.Sequential()
    # 卷积层部分
    for i, (num_convs, in_channels, out_channels) in enumerate(conv_arch):
        # 每经过一个vgg_block都会使宽高减半
        net.add_module("vgg_block_" + str(i+1), vgg_block(num_convs, in_channels, out_channels))
    # 全连接层部分
    net.add_module("fc", nn.Sequential(d2l.FlattenLayer(),
                                 nn.Linear(fc_features, fc_hidden_units),
                                 nn.ReLU(),
                                 nn.Dropout(0.5),
                                 nn.Linear(fc_hidden_units, fc_hidden_units),
                                 nn.ReLU(),
                                 nn.Dropout(0.5),
                                 nn.Linear(fc_hidden_units, 10)
                                ))
    return net


In [ ]:
net = vgg(conv_arch, fc_features, fc_hidden_units)
X = torch.rand(1, 1, 224, 224)

# named_children获取一级子模块及其名字(named_modules会返回所有子模块,包括子模块的子模块)
for name, blk in net.named_children():
    X = blk(X)
    print(name, 'output shape: ', X.shape)

### 网络中的网络
__LeNet、AlexNet和VGG__：先以由卷积层构成的模块充分抽取 空间特征，再以由全连接层构成的模块来输出分类结果。

__NiN__：
串联多个由卷积层和“全连接”层构成的小⽹络来构建⼀个深层⽹络。
⽤了输出通道数等于标签类别数的NiN块，然后使⽤全局平均池化层对每个通道中所有元素求平均并直接⽤于分类。

![Image Name](Pics/NIN.png)

1×1卷积核作用:

- 放缩通道数：通过控制卷积核的数量达到通道数的放缩。
- 增加非线性。1×1卷积核的卷积过程相当于全连接层的计算过程，并且还加入了非线性激活函数，从而可以增加网络的非线性。
- 充当全连接层的作用，减少卷积层到全连接层之间的显式数据转换（4维张量的flatten操作）



In [ ]:
## NIN 网络的简单实现

def nin_block(in_channels, out_channels, kernel_size, stride, padding):
    blk = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
                        nn.ReLU(),
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU(),
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU())
    return blk

class GlobalAvgPool2d(nn.Module):
    # 全局平均池化层可通过将池化窗口形状设置成输入的高和宽实现
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()
    def forward(self, x):
        return F.avg_pool2d(x, kernel_size=x.size()[2:])

net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, stride=4, padding=0),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nin_block(96, 256, kernel_size=5, stride=1, padding=2),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nin_block(256, 384, kernel_size=3, stride=1, padding=1),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 10, kernel_size=3, stride=1, padding=1),
    GlobalAvgPool2d(),
    # 将四维的输出转成二维的输出，其形状为(批量大小, 10)
    d2l.FlattenLayer())


In [ ]:
X = torch.rand(1, 1, 224, 224)
for name, blk in net.named_children():
    X = blk(X)
    print(name, 'output shape: ', X.shape)

# GoogLenet
通过串联卷积模块和 Inception 模块来搭建网络结构。


__Inception 模块：__
- 采用多个不同分支来提取不同细粒度的特征，从左到右，提取的特征越来越抽象；
- 同时，为了能使得最后的特征能够拼接，需要对不同的分支采用不同的 padding 等操作，保证输出的长和宽不变；
- 其中为了减少计算参数，在中间两个分支都采用了 1\*1 的卷积层来减少输出通道，从而减少卷积核的通道数。
- 最后，通过控制不同分支的输出通道数来控制生成的 Inception 模块的复杂度

![googlenet.png](Pics/googlenet.png)


In [ ]:
class Inception(nn.Module):
    # c1 - c4为每条线路里的层的输出通道数
    def __init__(self, in_c, c1, c2, c3, c4):
        super(Inception, self).__init__()
        # 线路1，单1 x 1卷积层
        self.p1_1 = nn.Conv2d(in_c, c1, kernel_size=1)
        # 线路2，1 x 1卷积层后接3 x 3卷积层
        self.p2_1 = nn.Conv2d(in_c, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1 x 1卷积层后接5 x 5卷积层
        self.p3_1 = nn.Conv2d(in_c, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3 x 3最大池化层后接1 x 1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_c, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        return torch.cat((p1, p2, p3, p4), dim=1)  # 在通道维上连结输出

#### GoogLenet 的完整结构
![img](Pics/googlenet_full.png)

In [ ]:
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1),
                   nn.Conv2d(64, 192, kernel_size=3, padding=1),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b3 = nn.Sequential(Inception(192, 64, (96, 128), (16, 32), 32),
                   Inception(256, 128, (128, 192), (32, 96), 64),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b4 = nn.Sequential(Inception(480, 192, (96, 208), (16, 48), 64),
                   Inception(512, 160, (112, 224), (24, 64), 64),
                   Inception(512, 128, (128, 256), (24, 64), 64),
                   Inception(512, 112, (144, 288), (32, 64), 64),
                   Inception(528, 256, (160, 320), (32, 128), 128),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

b5 = nn.Sequential(Inception(832, 256, (160, 320), (32, 128), 128),
                   Inception(832, 384, (192, 384), (48, 128), 128),
                   d2l.GlobalAvgPool2d())

net = nn.Sequential(b1, b2, b3, b4, b5,
                    d2l.FlattenLayer(), nn.Linear(1024, 10))

net = nn.Sequential(b1, b2, b3, b4, b5, d2l.FlattenLayer(), nn.Linear(1024, 10))

X = torch.rand(1, 1, 96, 96)

for blk in net.children():
    X = blk(X)
    print('output shape: ', X.shape)

#batchsize=128
batch_size = 16
# 如出现“out of memory”的报错信息，可减小batch_size或resize
#train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

### Resnet

通常来说，深层神经网路的表示函数能包含浅层神经网络的所有表示函数，因此，我们希望能根据输入数据，在更多的可能性中寻找合适的映射函数。但是，在使用深层神经网络时，我们往往会遇到过拟合问题，即神经网络没有拟合数据的一般规律，而是拟合了这一组数据中的噪声，造成神经网络的泛化能力差。因此，为了解决能更好地应用深层网络，Resnet 提供了一种跳接的连接方式，其示意图如下：

![img](Pics/residual.png)

可以发现，其通过将输入恒等加入到输出，促使后层网络能直接获取输入的信息，变相说来，这种跳接给网络提供了一种选择: 是否需要采用这个新加入的神经层，其也就使得网络能根据输入数据在复杂和简单的网络结构中进行良好的切换，实验结构表明，使用了 Resnet 跳接的网络可以增加很多层而不容易产生过拟合。

除此之外， Residual 模块提供了在恒等变化的基础上捕捉变化的能力，促使网络表达更加细腻。


In [ ]:
class Residual(nn.Module):
    #可以设定输出通道数、是否使用额外的1x1卷积层来修改通道数以及卷积层的步幅。
    # 由于 resnet 是将结果相加，因此输入输出的形状需要一致，因此我们还需要用1*1卷积来统一可能变化的通道数
    def __init__(self, in_channels, out_channels, use_1x1conv=False, stride=1):
        super(Residual, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        return F.relu(Y + X)

### ResNet模型
- 卷积(64,7x7,3)
- 批量一体化
- 最大池化(3x3,2)

- 残差块x4 (通过步幅为2的残差块在每个模块之间减小高和宽)

- 全局平均池化

- 全连接

In [ ]:
net = nn.Sequential(
        nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

def resnet_block(in_channels, out_channels, num_residuals, first_block=False):
    if first_block:
        assert in_channels == out_channels # 第一个模块的通道数同输入通道数一致
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(in_channels, out_channels, use_1x1conv=True, stride=2))
        else:
            blk.append(Residual(out_channels, out_channels))
    return nn.Sequential(*blk)

net.add_module("resnet_block1", resnet_block(64, 64, 2, first_block=True))
net.add_module("resnet_block2", resnet_block(64, 128, 2))
net.add_module("resnet_block3", resnet_block(128, 256, 2))
net.add_module("resnet_block4", resnet_block(256, 512, 2))
net.add_module("global_avg_pool", d2l.GlobalAvgPool2d()) # GlobalAvgPool2d的输出: (Batch, 512, 1, 1)
net.add_module("fc", nn.Sequential(d2l.FlattenLayer(), nn.Linear(512, 10)))

In [ ]:
X = torch.rand((1, 1, 224, 224))
for name, layer in net.named_children():
    X = layer(X)
    print(name, ' output shape:\t', X.shape)

### DenseNet
densenet 通过将跳接之后的连接过程从相加修改为 concatenate，形成了一个更激进的密集连接机制：即互相连接所有的层，即每个层都会接受其前面所有层作为其额外的输入。这样一来，后层可以直接访问前面任意层特征图，促进了信息综合。

![img](Pics/densenet.png)

![img](Pics/densenetjump.png)

具体说来，其优势如下：
1. DenseNet提升了梯度的反向传播，使得网络更容易训练。由于每层可以直达最后的误差信号，实现了隐式的“deep supervision”；
2. 参数更小且计算更高效，其采用较小的growth rate，实现每个层所独有的特征图是比较小的；同时采用过渡层也减少了参数
3. 由于特征复用，最后的分类器使用了低级特征，最后决策也就有机会采用多个尺度的特征


